In [34]:
import warnings
import pandas as pd


#important converse imports.
from pyconverse import Callyzer
from pyconverse import SpeakerStats
from pyconverse import SemanticTextSegmention
from pyconverse import ZeroShotTopicFinder

warnings.filterwarnings('ignore')


#first time you install pyconverse and make these imports it downloads few transformers models in the backend, hence the first import might take few minutes.

In [2]:
df = pd.read_csv("../Converse/notebook/sample_1.csv") #read sample data

In [3]:
ca = Callyzer(data=df, speaker='who') #initialise the core call analysis class.

In [4]:
ca.get_interruption() #compute basic stats like interruptions

{'CHI': {'metadata': [{'start_time': 1222.365,
    'end_time': 1223.032,
    'index': 200}],
  'count': 1},
 'INV': {'metadata': [{'start_time': 574.198,
    'end_time': 574.931,
    'index': 102}],
  'count': 1},
 'total_interruption': 2}

In [5]:
ca.get_silence() #compute basic stats like periods of silence

{'CHI': {'metadata': [{'start_time': 188.765,
    'end_time': 189.865,
    'index': 34},
   {'start_time': 275.082, 'end_time': 279.215, 'index': 48},
   {'start_time': 3182.483, 'end_time': 3190.575, 'index': 617}],
  'count': 3},
 'INV': {'metadata': [{'start_time': 16.161, 'end_time': 20.777, 'index': 3},
   {'start_time': 963.0, 'end_time': 964.2, 'index': 166},
   {'start_time': 1509.049, 'end_time': 1512.706, 'index': 253},
   {'start_time': 1820.52, 'end_time': 1822.656, 'index': 315},
   {'start_time': 3485.7, 'end_time': 3486.594, 'index': 685},
   {'start_time': 3539.524, 'end_time': 3540.255, 'index': 700}],
  'count': 6},
 'total_interruption': 9}

In [7]:
temp = ca.tag_backchannel() #identify backchannel utterance. 
temp.query("is_backchannel == True")

,filename,path,utt_num,who,role,postcodes,gems,utterance,startTime,endTime,is_backchannel
28,14,class/Bradford/14,29,INV,Investigator,NaN,NaN,okay,156.133,156.866,True
52,14,class/Bradford/14,55,INV,Investigator,NaN,NaN,okay,284.348,284.914,True
65,14,class/Bradford/14,68,INV,Investigator,NaN,NaN,okay,342.564,343.030,True
83,14,class/Bradford/14,86,INV,Investigator,NaN,NaN,okay,450.400,451.066,True
105,14,class/Bradford/14,108,INV,Investigator,NaN,NaN,okay,585.980,586.630,True
120,14,class/Bradford/14,123,INV,Investigator,NaN,NaN,okay,701.130,701.530,True
128,14,class/Bradford/14,131,INV,Investigator,NaN,NaN,okay,741.783,742.149,True
135,14,class/Bradford/14,138,CHI,Child,NaN,NaN,no,770.545,771.411,True
136,14,class/Bradford/14,139,CHI,Child,NaN,NaN,yeah,771.411,771.895,True
158,14,class/Bradford/14,161,INV,Investigator,NaN,NaN,okay,916.974,917.574,True


In [8]:
# here we use nlp mode to identify backchannels . Inspired form this paper: https://arxiv.org/pdf/2106.12978.pdf 
# We take these -> [okay, thats it, ummhhh] as backchannel, add max pool and then we apply sentence similarity with all utterances.
temp2 = ca.tag_backchannel(type='nlp')
temp2.query("is_backchannel == True")

/Users/rita/opt/anaconda3/lib/python3.8/site-packages/transformers/configuration_utils.py:336: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
/Users/rita/opt/anaconda3/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


,filename,path,utt_num,who,role,postcodes,gems,utterance,startTime,endTime,is_backchannel
17,14,class/Bradford/14,18,INV,Investigator,NaN,NaN,okay great,79.315,81.382,True
21,14,class/Bradford/14,22,CHI,Child,NaN,NaN,well he,103.298,114.700,True
28,14,class/Bradford/14,29,INV,Investigator,NaN,NaN,okay,156.133,156.866,True
37,14,class/Bradford/14,39,INV,Investigator,NaN,NaN,okay good,196.855,198.232,True
52,14,class/Bradford/14,55,INV,Investigator,NaN,NaN,okay,284.348,284.914,True
...,...,...,...,...,...,...,...,...,...,...,...
667,14,class/Bradford/14,675,INV,Investigator,NaN,NaN,that's it,3423.566,3424.170,True
681,14,class/Bradford/14,689,INV,Investigator,NaN,NaN,mhm.,3460.900,3461.353,True
685,14,class/Bradford/14,694,INV,Investigator,NaN,NaN,okay,3485.700,3486.594,True
696,14,class/Bradford/14,705,INV,Investigator,NaN,NaN,okay,3532.419,3533.789,True


In [9]:
# try to identify the interrogative questions
ca.tag_questions().query("is_question == True")

,filename,path,utt_num,who,role,postcodes,gems,utterance,startTime,endTime,is_backchannel,is_question
84,14,class/Bradford/14,87,INV,Investigator,NaN,NaN,what else.,451.066,451.916,False,True
167,14,class/Bradford/14,171,INV,Investigator,NaN,NaN,what else can you tell me.,964.200,966.666,False,True
446,14,class/Bradford/14,453,INV,Investigator,NaN,NaN,what do you know about ballet.,2382.482,2384.188,False,True
474,14,class/Bradford/14,481,INV,Investigator,NaN,NaN,what do you know about the Blues.,2539.778,2541.891,False,True
572,14,class/Bradford/14,579,INV,Investigator,NaN,NaN,what do you know about the Columbine shooting.,2990.690,2993.140,False,True
578,14,class/Bradford/14,585,INV,Investigator,NaN,NaN,what do you know about the Oklahoma_City bombing.,3018.327,3021.090,False,True
585,14,class/Bradford/14,592,INV,Investigator,NaN,NaN,what do you know about Princess_Diana.,3038.958,3040.908,False,True
616,14,class/Bradford/14,623,INV,Investigator,NaN,NaN,what do you know about John_F_Kennedy_Junior.,3172.962,3175.517,False,True
663,14,class/Bradford/14,671,INV,Investigator,NaN,NaN,what do you know about the Million_Man_March.,3393.043,3395.748,False,True
668,14,class/Bradford/14,676,INV,Investigator,NaN,NaN,what do you know about Moesha.,3424.170,3425.586,False,True


In the customer call center, people like to observe if the customer is angry, frustrated, etc, when we apply for bank customer service center, to see if the agent is responding with empathy or not when we identify the vulerability. If customer is vulnerable (like suffering from health issue, old age), its expected to reply with empathy. 


let's see how we could do these with converse. 

In [40]:
# convert the data at speak at turn level
df = ca.convert_at_turn()
df.head()

,filename,path,utt_num,who,role,postcodes,gems,utterance,startTime,endTime,is_backchannel,is_question,turn_id
0,14,class/Bradford/14,0,CHI,Child,NaN,NaN,he wan an explorer and he discovered America a...,0.0,7.983,False,False,0
3,14,class/Bradford/14,4,INV,Investigator,NaN,NaN,tell me what you know about the Pilgrims.,16.161,20.777,False,False,1
4,14,class/Bradford/14,5,CHI,Child,NaN,NaN,well they moved from England and came to Ameri...,20.777,46.4,False,False,2
10,14,class/Bradford/14,11,INV,Investigator,NaN,NaN,can you tell me anything else about the Pilgrims.,46.4,50.516,False,False,3
11,14,class/Bradford/14,12,CHI,Child,NaN,NaN,they moved from England because of the king he...,50.516,77.715,False,False,4


In [41]:
# Identiy the overall nature of the speakers.
ss = SpeakerStats(df, speaker='who')
ss.get_stats()

{'CHI': ['Focused on the past', 'Verbal fluency, cognitive complexity'],
 'INV': ['Use of concrete nouns, interest in objects/things',
  'Education, concern with precision']}

In [42]:
# lets segment our calls into bigger chunks of texts via semantic sentence similairty & text tilling algorithms. 
sts = SemanticTextSegmention(df)
segments = sts.get_segments()

In [43]:
### Identify topics being discussed in a call via zero shot topic infernce
zst = ZeroShotTopicFinder()

In [44]:
text = segments[1]
print(f"Text: {text}")
print(f"Topics: {zst.find_topic(text)}")

Text:  can you tell me anything else okay great tell me everything you know about George_Washington. he was the first president he was well he I'm trying to well he fought in the Civil_War he was a general in the Civil_War and chopped down his father's cherry tree when he was a little boy he that's it.
Topics: ['War', 'Head Of State']
